# Quantum Integer Programming (QuIP) 47-779. Fall 2020, CMU

This notebook contains material from the Quantum Integer Programming Lecture at CMU Fall 2020 by David Bernal (bernalde at cmu.edu), Sridhar Tayur (stayur at cmu.edu) and Davide Venturelli; the content is available on **[Github](https://github.com/bernalde/QuIP)**. The text is released under the **[CC-BY-NC-ND-4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode) license, and code is released under the **[MIT license](https://opensource.org/licenses/MIT).*

Run in **[Google Colab](https://colab.research.google.com/github/bernalde/QuIP/blob/master/notebooks/Notebook%20Quiz%201.ipynb)**

## Quiz 1 Mathematical programming
Here we will model an exercise given in the lecture. This example will use as modeling language **[Pyomo](http://www.pyomo.org/)**, an open-source Python package, which provides a flexible access to different solvers and a general modeling framework for linear and nonlinear integer programs.
The examples solved here will make use of open-source solvers **[CLP/CBC](https://projects.coin-or.org/Cbc)** for linear and mixed-integer linear programming, **[BONMIN](https://www.coin-or.org/Bonmin/)** for convex integer nonlinear programming and **[COUENNE](https://projects.coin-or.org/Couenne)** for nonconvex (global) integer nonlinear programming.

### Problem statement
#### Integer linear program
Solve the following problem
$$
\min_{\mathbf{x}} 2𝑥_0+4𝑥_1+4𝑥_2+4𝑥_3+4𝑥_4+4𝑥_5+5𝑥_6+4𝑥_7+5𝑥_8+6𝑥_9+5𝑥_{10} \\
s.t. \begin{bmatrix}
1 & 0 & 0 & 1 & 1 & 1 & 0 & 1 & 1 & 1 & 1\\
0 & 1 & 0 & 1 & 0 & 1 & 1 & 0 & 1 & 1 & 1\\
0 & 0 & 1 & 0 & 1 & 0 & 1 & 1 & 1 & 1 & 1
\end{bmatrix}\mathbf{x}=
\begin{bmatrix}
1\\
1\\
1
\end{bmatrix} \\
\mathbf{x} \in \{0,1 \}^{11}
$$
Equivalently written as

$$
\min_{\mathbf{x}} \mathbf{c}^\top \mathbf{x}\\
s.t. \mathbf{A}\mathbf{x}=\mathbf{b} \\
\mathbf{x} \in \{0,1 \}^{11}
$$

In [1]:
# If using this on Google collab, we need to install the packages
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Let's start with Pyomo
if IN_COLAB:
    !pip install -q pyomo

In [2]:
# Import the Pyomo library, which can be installed via pip, conda or from Github https://github.com/Pyomo/pyomo
import pyomo.environ as pyo
# Import Matplotlib to generate plots
import matplotlib.pyplot as plt
# Import numpy and scipy for certain numerical calculations below
import numpy as np
from scipy.special import gamma
import math

In [3]:
# Define the model
model = pyo.ConcreteModel(name='Quiz 1, 47-779 QuIP')
# Define the problem data
A = np.array([[1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
            [0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1],
            [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]])
b = np.array([1, 1, 1])
c = np.array([2, 4, 4, 4, 4, 4, 5, 4, 5,6, 5])

# set of row indices
I = range(len(A))
# set of column indices
J = range(len(A.T))

#Define the variables (here we have everything using Python's 0-based counting, sorry)
model.x = pyo.Var(J, domain=pyo.Binary)

# add model constraints
model.constraints = pyo.ConstraintList()
for i in I:
    model.constraints.add(sum(A[i,j]*model.x[j] for j in J) == b[i])

# Define the objective function
model.objective = pyo.Objective(expr = sum(c[j]*model.x[j] for j in J), sense=pyo.minimize)
# Print the model
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : 

In [4]:
# Let's install the LP/MIP solver CBC
if IN_COLAB:
    !apt-get install -y -qq coinor-cbc

In [5]:
# Define the solver GLPK
if IN_COLAB:
    opt_cbc = pyo.SolverFactory('cbc', executable='/usr/bin/cbc')
else:
    opt_cbc = pyo.SolverFactory('cbc')
# Here we could use another solver, e.g. gurobi or cplex
# opt_gurobi = pyo.SolverFactory('gurobi')

In [6]:
# We solve the problem with CBC
result_obj = opt_cbc.solve(model, tee=False)
model.display()

Model Quiz 1, 47-779 QuIP

  Variables:
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary
          9 :     0 :   0.0 :     1 : False : False : Binary
         10 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   5.0

  Constraints:
    constraints : Size=3
        Key : Lower : Body : Uppe

We observe that the optimal solution of this problem is $x_{10} = 1, 0$ otherwise, leading to an objective of $5$. Notice that this problem has a degenerate optimal solution given that $x_8 = 1, 0$ otherwise also leads to the same solution.



### Integer convex nonlinear programming
Now solve
$$
\max_{\mathbf{x}} \sum_i exp(c_i x_i^2)\\
s.t. \mathbf{A}\mathbf{x}=\mathbf{b} \\
\mathbf{x} \in \{0,1 \}^{11}
$$

In [7]:
# Define the model
model_icnlp = pyo.ConcreteModel(name='Quiz 1 convex nonlinear, 47-779 QuIP')

#Define the variables (here we have everything using Python's 0-based counting, sorry)
model_icnlp.x = pyo.Var(J, domain=pyo.Binary)

# add model constraints
model_icnlp.constraints = pyo.ConstraintList()
for i in I:
    model_icnlp.constraints.add(sum(A[i,j]*model_icnlp.x[j] for j in J) == b[i])

# Define the objective function
model_icnlp.objective = pyo.Objective(expr = sum(pyo.exp(c[j]*model_icnlp.x[j]*model_icnlp.x[j]) for j in J), sense=pyo.minimize)
# Print the model
model_icnlp.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : 

In [8]:
# Define the solver BONMIN
if IN_COLAB:
    !wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
    !unzip -o -q bonmin-linux64
    
opt_bonmin = pyo.SolverFactory('bonmin', executable='/content/bonmin')

In [9]:
# Here we solve the optimization problem, the option tee=True prints the solver output
result_obj_icnlp = opt_bonmin.solve(model_icnlp, tee=False)
model_icnlp.display()

Model Quiz 1 convex nonlinear, 47-779 QuIP

  Variables:
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary
          9 :     0 :   0.0 :     1 : False : False : Binary
         10 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 118.19630006628847

  Constraints:
    constraints : Size=3
   

In this case the optimal solution becomes $x_2 = 1, x_5 = 1$ with an objective of $118.2$. This solution is also degenerate.

### Integer non-convex programming
Now solve
$$
\max_{\mathbf{x}} \sum_i log(c_i + x_i)\\
s.t. \mathbf{A}\mathbf{x}=\mathbf{b} \\
\mathbf{x} \in \{0,1 \}^{11}
$$

In [10]:
# Define the model
model_inlp = pyo.ConcreteModel(name='Quiz 1 convex nonlinear, 47-779 QuIP')

#Define the variables (here we have everything using Python's 0-based counting, sorry)
model_inlp.x = pyo.Var(J, domain=pyo.Binary)

# add model constraints
model_inlp.constraints = pyo.ConstraintList()
for i in I:
    model_inlp.constraints.add(sum(A[i,j]*model_inlp.x[j] for j in J) == b[i])

# Define the objective function
model_inlp.objective = pyo.Objective(expr = sum(pyo.log(c[j] + model_inlp.x[j]) for j in J), sense=pyo.minimize)
# Print the model
model_inlp.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : 

In [11]:
# Trying to solve the problem with BONMIN we might obtain the optimal solution, but we have no guarantees
result_obj_inlp = opt_bonmin.solve(model_inlp, tee=False)
model_inlp.display()

Model Quiz 1 convex nonlinear, 47-779 QuIP

  Variables:
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary
          9 :     0 :   1.0 :     1 : False : False : Binary
         10 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 15.785137233636902

  Constraints:
    constraints : Size=3
   

In [12]:
# Define the solver COUENNE
if IN_COLAB:
    !wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
    !unzip -o -q couenne-linux64
    
opt_couenne = pyo.SolverFactory('couenne', executable='/content/couenne')

In [13]:
# Trying to solve the problem with global MINLP solver COUENNE
result_obj_ncinlp = opt_couenne.solve(model_inlp, tee=False)
model_inlp.display()

Model Quiz 1 convex nonlinear, 47-779 QuIP

  Variables:
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary
          9 :     0 :   1.0 :     1 : False : False : Binary
         10 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 15.785137233636902

  Constraints:
    constraints : Size=3
   

In this case the unique optimal solution becomes $x_9 = 1$ with an objective of $15.8$.